In [ ]:

!pip3 install --upgrade gdown --quiet
#!gdown 16GYHdSWS3iMYwMPv5FpeDZN2rH7PR0F2 # this is the file ID of miniplaces dataset
!gdown 1YXp7HwhtdkU7F11QTZXsTPrT9jq3LDAn
# back-up commands (try the following it previous file id is overload)
# !gdown 1CyIQOJienhNITwGcQ9h-nv8z6GOjV2HX


Downloading...
From: https://drive.google.com/uc?id=1YXp7HwhtdkU7F11QTZXsTPrT9jq3LDAn
To: /content/dataset.tar.gz
100% 28.5M/28.5M [00:00<00:00, 152MB/s]


In [ ]:
import os
import tarfile
from tqdm import tqdm
import urllib.request

def setup(folder_name='Week7'):
  # Let's make our assignment directory
  CS188_path = './'
  os.makedirs(os.path.join(CS188_path, 'Week7', 'data'), exist_ok=True)
  # Now, let's specify the assignment path we will be working with as the root.
  root_dir = os.path.join(CS188_path, 'Week7')
  # Open the tar.gz file
  tar = tarfile.open("dataset.tar.gz", "r:gz")
  # Extract the file "./Assignment2/data" folder
  total_size = sum(f.size for f in tar.getmembers())
  with tqdm(total=total_size, unit="B", unit_scale=True, desc="Extracting tar.gz file") as pbar:
      for member in tar.getmembers():
          tar.extract(member, os.path.join(root_dir, 'data'))
          pbar.update(member.size)
  # Close the tar.gz file
  tar.close()
  return root_dir

In [ ]:

root_dir = setup(folder_name='Week7')

Extracting tar.gz file: 100%|██████████| 28.4M/28.4M [00:01<00:00, 16.8MB/s]


### Define the data transform


In [ ]:
from torchvision import transforms

# Define data transformation
# You can copy your data transform from Assignment2. 
# Notice we are resize images to 128x128 instead of 64x64.
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomCrop((96,96)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


### Define the dataset and dataloader

In [ ]:

import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class Metadrive(Dataset):
    def __init__(self, root_dir, split, transform=None):
        
        assert split in ['train', 'val', 'test']
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.filenames = []
        self.steering_angle = {0: [], 1: [], 2: []} #0:left 1:right 2:straight
        self.accel = {0: [], 1: [], 2: []} #0:left 1:right 2:straight
        self.label = []

        count = 0
        # Iterate directory
        for path in os.listdir(os.path.join(root_dir, 'dataset', split)):
          if os.path.isfile(os.path.join(root_dir, 'dataset', split, path)):
            self.filenames.append(path)

        for filename in self.filenames:
          filename = filename[1:-5]
          filename = filename.split(', ')
          if float(filename[0])*100 > 10:
            category = 0
          elif float(filename[0])*100 < -10:
            category = 1
          else:
            category = 2
          self.label.append(category)
          self.steering_angle[category].append(float(filename[0]))
          self.accel[category].append(float(filename[1]))

    def __len__(self):
        
        return len(self.filenames)

    def __getitem__(self, idx):
        '''
        Label returned is of the form: tuple((float)steering, (float)acceleration, (float)velocity)
        '''

        image_path = os.path.join(self.root_dir, 'dataset', self.split, self.filenames[idx])
        image = Image.open(image_path)
        if self.transform:
          image = self.transform(image)
        return image, self.label[idx]

    def get_average(self, category):
      return sum(self.steering_angle[category])/len(self.steering_angle[category]), sum(self.accel[category])/len(self.accel[category])

### Define the train method

In [ ]:
def train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs):
    """
    Train the MLP classifier on the training set and evaluate it on the validation set every epoch.
    
    Args:
        model (MLP): MLP classifier to train.
        train_loader (torch.utils.data.DataLoader): Data loader for the training set.
        val_loader (torch.utils.data.DataLoader): Data loader for the validation set.
        optimizer (torch.optim.Optimizer): Optimizer to use for training.
        criterion (callable): Loss function to use for training.
        device (torch.device): Device to use for training.
        num_epochs (int): Number of epochs to train the model.
    """
    # Place model on device
    model = model.to(device)

    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.001)
    
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        # Use tqdm to display a progress bar during training
        with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}') as pbar:
            for inputs, labels in train_loader:
                # Move inputs and labels to device
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero out gradients
                optimizer.zero_grad()
                
                # Compute the logits and loss
                logits = model(inputs)
                loss = criterion(logits, labels)
                
                # Backpropagate the loss
                loss.backward()
                
                # Update the weights
                optimizer.step()
                
                # Update the progress bar
                pbar.update(1)
                pbar.set_postfix(loss=loss.item())
          
        #scheduler.step()
        
        # Evaluate the model on the validation set
        avg_loss, accuracy = evaluate(model, val_loader, criterion, device)
        print(f'Validation set: Average loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}')

def evaluate(model, test_loader, criterion, device):
    """
    Evaluate the MLP classifier on the test set.
    
    Args:
        model (MLP): MLP classifier to evaluate.
        test_loader (torch.utils.data.DataLoader): Data loader for the test set.
        criterion (callable): Loss function to use for evaluation.
        device (torch.device): Device to use for evaluation.
        
    Returns:
        float: Average loss on the test set.
        float: Accuracy on the test set.
    """
    model.eval()  # Set model to evaluation mode
    
    with torch.no_grad():
        total_loss = 0.0
        num_correct = 0
        num_samples = 0
        
        for inputs, labels in test_loader:
            # Move inputs and labels to device
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Compute the logits and loss
            logits = model(inputs)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            # Compute the accuracy
            _, predictions = torch.max(logits, dim=1)
            num_correct += (predictions == labels).sum().item()
            num_samples += len(inputs)
            
            
    # Compute the average loss and accuracy
    avg_loss = total_loss / len(test_loader)
    accuracy = num_correct / num_samples
    
    return avg_loss, accuracy

In [ ]:
# Also, seed everything for reproducibility
# code from https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964#file-seed_everything-py
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
# Define the device to use for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == torch.device('cuda'):
    print(f'Using device: {device}. Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

Using device: cuda. Good to go!


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import cv2
import torchvision.models as models
class Resnet(nn.Module):
    def __init__(self, mode='linear',pretrained=True):
        super().__init__()
        """
        use the resnet18 model from torchvision models. Remember to set pretrained as true
        
        mode has three options:
        1) features: to extract features only, we do not want the last fully connected layer of 
            resnet18. Use nn.Identity() to replace this layer.
        2) linear: For this model, we want to freeze resnet18 features, then train a linear 
            classifier which takes the features before FC (again we do not want 
            resnet18 FC). And then write our own FC layer: which takes in the features and 
            output scores of size 100 (because we have 100 categories).
            Because we want to freeze resnet18 features, we have to iterate through parameters()
            of our model, and manually set some parameters to requires_grad = False
            Or use other methods to freeze the features
        3) finetune: Same as 2), except that we we do not need to freeze the features and
           can finetune on the pretrained resnet model.
        """
        self.resnet = None
        self.resnet = models.resnet18(pretrained = pretrained)

        if mode == 'feature':
          self.resnet.fc = nn.Identity()
        
        if mode == 'linear':
          for param in self.resnet.parameters():
            param.requires_grad = False
          self.resnet.fc = nn.Linear(512, 3)

        if mode == 'finetune':
          for param in self.resnet.parameters():
            param.requires_grad = True
          self.resnet.fc = nn.Linear(512, 3)
    #####################################################################################

    def forward(self, x):

        return self.resnet(x)
    
    def to(self,device):
        return self.resnet.to(device=device)


In [ ]:
seed_everything(0)

# Define the model, optimizer, and criterion (loss_fn)
model = Resnet(mode='finetune',pretrained=True)

# optimizer = torch.optim.SGD(
#     model.parameters(), 
#     lr=0.001, 
#     momentum=0.9)

optimizer = torch.optim.SGD(model.resnet.fc.parameters(), lr=0.001, momentum=0.9)
for name, param in model.named_parameters():
    if param.requires_grad and 'fc' not in name:
        optimizer.add_param_group({'params': param, 'lr':0.0001})

criterion = nn.CrossEntropyLoss()

# Define the dataset and data transform with flatten functions appended
data_dir = os.path.join(root_dir, 'data')

train_dataset = Metadrive(
    root_dir=data_dir, split='train', 
    transform=data_transform)

val_dataset = Metadrive(
    root_dir=data_dir, split='val', 
    transform=data_transform)

# Define the batch size and number of workers
batch_size = 64
num_workers = 2

# Define the data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

# Train the model
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=9)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/9: 100%|██████████| 15/15 [00:01<00:00, 10.32it/s, loss=0.415]


Validation set: Average loss = 0.6001, Accuracy = 0.8636


Epoch 2/9: 100%|██████████| 15/15 [00:01<00:00, 10.57it/s, loss=0.446]


Validation set: Average loss = 0.5048, Accuracy = 0.8409


Epoch 3/9: 100%|██████████| 15/15 [00:01<00:00, 10.41it/s, loss=0.411]


...

Validation set: Average loss = 0.3705, Accuracy = 0.8750


Epoch 8/9: 100%|██████████| 15/15 [00:01<00:00, 10.70it/s, loss=0.289]


Validation set: Average loss = 0.3555, Accuracy = 0.8750


Epoch 9/9: 100%|██████████| 15/15 [00:01<00:00, 10.77it/s, loss=0.308]


Validation set: Average loss = 0.3545, Accuracy = 0.8864


In [ ]:
import matplotlib.pyplot as plt

for path in os.listdir(os.path.join(data_dir, 'dataset', 'val')):
    if os.path.isfile(os.path.join(data_dir, 'dataset', 'val', path)):
      image_path = os.path.join(data_dir, 'dataset', 'val', path)

      image_path = "/content/Week7/data/dataset/val/(0.10624197352048775, -0.0027929785415545805, 29.14923399838864).png"

      image = Image.open(image_path)

      data_transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.CenterCrop((96,96)),
          transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
      ])

      image = data_transform(image).cuda()

      image = torch.unsqueeze(image, 0)

      res = model(image)



      print(res)

      break

tensor([[ 0.6871, -2.7549,  0.5548]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [ ]:
PATH = "model_categorize_revised.pt"
# torch.save({
#     # 'epoch': EPOCH,
#     'model_state_dict': model.state_dict(),
#     # 'optimizer_state_dict': optimizer.state_dict(),
#     # 'loss': LOSS,
# }, PATH)

torch.save(model.state_dict(), PATH)

In [ ]:
print(train_dataset.get_average(0))
print(train_dataset.get_average(1))
print(train_dataset.get_average(2))


(0.12388312854632436, 0.2091152917264203)
(-0.12438725769093287, 0.34663663748581514)
(-0.00500231837265083, 0.3247432458997098)
